# JUICE RPWI HF SID23 (PSSR3 rich): L1a QL -- 2023/12/24

# Import lib

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

# CDF and Directory setting: set by User

In [ ]:
# The latest CDF library https://spdf.gsfc.nasa.gov/pub/software/cdf/dist/latest/
from spacepy import pycdf

import os
os.environ["CDF_LIB"] = "/Applications/cdf/cdf39_0-dist/lib"

# *** Library ***
sys.path.append('./lib/')
import juice_cdf_lib as juice_cdf
import juice_math_lib as juice_math
import juice_sid23_lib as juice_sid23

# Setting and Read CDF file: set by User

In [ ]:
# *** Mode set ***
band_mode = 0                           # 0: sum    1: /Hz
unit_mode = 0                           # 0: raw    1: dBm＠ADC  2: V@HF   3:V2@HF   4:V2@RWI
f_mode = 0                              # 0: linear   1:log  in frequency
dump_mode = 1                           # 0: no-dump  1:plot dump

# *** Parameter set ***
cal = 0                                 # 0: background     1: CAL
p_raw_max = 8.5                         # background: 7.5   CAL: 10
p_raw_min = 1.0                         # background: 2.5   CAL: 5

# *** Conversion factor: cal_mode ***
cf, p_max, p_min = juice_cdf.cal_factors(band_mode, unit_mode, cal, p_raw_max, p_raw_min)
print("conversion factor:", cf, "    MAX-min:", p_max, p_min)

In [ ]:
# *** Directory set: set by User ***
work_dir = '/Users/user/0-python/JUICE_python/ql/'   # Plot dump folder

# *** Ver.2 ***
# 202310 -- SAMPLE
data_dir = '/Users/user/0-python/JUICE_data/test-CCSDS/sample/cdf/'        # CDF data folder
data_name = 'SID23_20231024-0011.cdf'
data_name = 'SID23_20231024-0024.cdf'
data_name = 'SID23_20231024-0034.cdf'

# 202311 -- FS
data_dir = '/Users/user/0-python/JUICE_data/test-TMIDX/202311_FS/cdf/'        # CDF data folder
data_name = 'SID23_Seq07.cdf'
#data_name = 'SID23_Seq08.cdf'
#data_name = 'SID23_Seq09.cdf'

# get CDF data

In [ ]:
cdf_file = data_dir + data_name
%ls $cdf_file

cdf = pycdf.CDF(cdf_file)
data = juice_sid23.juice_getdata_hf_sid23(cdf, cf)

date1 = data.epoch[0]
date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[-1]
date2 = date2.strftime('%Y/%m/%d %R:%S')
str_date = date1 + "  -  " + date2
print(str_date)

n_time0 = data.n_time
n_block0 = data.time.shape[1]
n_length0 = data.time.shape[2]
n_num0 = n_block0 * n_length0 

In [ ]:
# Mode 
N_ch = data.U_selected[0] + data.V_selected[0] + data.W_selected[0]
print("Ch:", N_ch, "  (U:", data.U_selected[0], "  V:", data.V_selected[0], "  W:", data.W_selected[0], ")")
print("==> Num-samples:", n_time0, "   Num-Block:", n_block0, " * Length:", n_length0, " = Num:", n_num0)
print("    time_length:", data.time[0][0][n_length0-1], data.time[0][1][0], "sec in 1-block", data.time[0][n_block0-1][n_length0-1], "sec in total")

# Raw data

In [ ]:
fig = plt.figure(figsize=(12, 16))
ax1 = fig.add_subplot(5, 1, 1);  ax3 = fig.add_subplot(5, 1, 2);  ax5 = fig.add_subplot(5, 1, 3)
ax7 = fig.add_subplot(5, 1, 4);  ax8 = fig.add_subplot(5, 1, 5);  

ax1.plot(np.ravel(data.Eu_i[:][:]), '-b', linewidth=.5, label='Eu_i'); ax1.plot(np.ravel(data.Eu_q[:][:]), ':r', linewidth=.5, label='Eu_q')
ax3.plot(np.ravel(data.Ev_i[:][:]), '-b', linewidth=.5, label='Ev_i'); ax3.plot(np.ravel(data.Ev_q[:][:]), ':r', linewidth=.5, label='Ev_q')
ax5.plot(np.ravel(data.Ew_i[:][:]), '-b', linewidth=.5, label='Ew_i'); ax5.plot(np.ravel(data.Ew_q[:][:]), ':r', linewidth=.5, label='Ew_q')
ax7.plot(np.ravel(data.time[:]), linewidth=.5, label='time')
ax8.plot(np.ravel(data.epoch[:]), '.')
ax1.set_ylabel('Eu');  ax3.set_ylabel('Ev');  ax5.set_ylabel('Ew');   ax7.set_ylabel('Time [sec]')
ax8.set_xlabel(str_date)
#
title_label = '[JUICE/RPWI HF PSSR3-Rich (SID-23)]  ' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8)
ax7.legend(loc='upper right', fontsize=8)

# range: X-axis
"""
xlim=[0, len(np.ravel(data.Eu_i[:][:]))];  print(xlim)
ax1.set_xlim(xlim);  ax3.set_xlim(xlim);  ax5.set_xlim(xlim);  ax7.set_xlim(xlim)
"""
# range: Y-axis
"""
ylim=[-10**(p_raw_max-3.5), 10**(p_raw_max-3.5)]
ax1.set_ylim(ylim);  ax3.set_ylim(ylim);  ax5.set_ylim(ylim)
"""

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw.png'
    fig.savefig(png_fname)

## RAW - 1 Sweep

In [ ]:
n_sweep = np.int16(n_time0/2)
n_sweep = n_time0-5
n_block = n_block0
print("[specific sweep]  SWEEP:", n_sweep, "in total of", n_time0, "   Block:", n_block, "in total of", data.N_block[0])

fig = plt.figure(figsize=(12, 10))
ax1 = fig.add_subplot(4, 1, 1);  ax3 = fig.add_subplot(4, 1, 2);  ax5 = fig.add_subplot(4, 1, 3);  ax7 = fig.add_subplot(4, 1, 4);  

ax1.plot(np.ravel(data.Eu_i[n_sweep][0:n_block]), '-b', linewidth=.5, label='Eu_i'); 
ax1.plot(np.ravel(data.Eu_q[n_sweep][0:n_block]), ':r', linewidth=.5, label='Eu_q')
ax3.plot(np.ravel(data.Ev_i[n_sweep][0:n_block]), '-b', linewidth=.5, label='Ev_i'); 
ax3.plot(np.ravel(data.Ev_q[n_sweep][0:n_block]), ':r', linewidth=.5, label='Ev_q')
ax5.plot(np.ravel(data.Ew_i[n_sweep][0:n_block]), '-b', linewidth=.5, label='Ew_i'); 
ax5.plot(np.ravel(data.Ew_q[n_sweep][0:n_block]), ':r', linewidth=.5, label='Ew_q')
ax7.plot(np.ravel(data.time[n_sweep][0:n_block]), linewidth=.5, label='time')
ax1.set_ylabel('Eu');  ax3.set_ylabel('Ev');  ax5.set_ylabel('Ew');    ax7.set_ylabel('Time [sec]')
#
title_label = '[JUICE/RPWI HF PSSR3-Rich (SID-23)]  ' + data_name + "  [" + date1 + "]";  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8)
ax7.legend(loc='upper right', fontsize=8)

# range: X-axis
"""
xlim=[0, n_block*data.N_feed[0]*128]
ax1.set_xlim(xlim);  ax3.set_xlim(xlim);  ax5.set_xlim(xlim);  ax7.set_xlim(xlim)
"""
# range: Y-axis
"""
ylim=[-10**(p_raw_max-3.5), 10**(p_raw_max-3.5)]
ax1.set_ylim(ylim);  ax3.set_ylim(ylim);  ax5.set_ylim(ylim)
"""

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw-sweep.png'
    fig.savefig(png_fname)

## RAW - 1 block

In [ ]:
n_sweep = np.int16(n_time0/2)
n_sweep = n_time0-5
n_block = 1
print("[specific block]  SWEEP:", n_sweep, "in total of ", n_time0, "   Block:", n_block, "in total of ", data.N_block[0])

fig = plt.figure(figsize=(12, 10))
ax1 = fig.add_subplot(3, 1, 1);  ax3 = fig.add_subplot(3, 1, 2);  ax5 = fig.add_subplot(3, 1, 3)

ax1.plot(np.ravel(data.time[n_sweep][0:n_block]), np.ravel(data.Eu_i[n_sweep][0:n_block]), '-b', linewidth=.5, label='Eu_i'); 
ax1.plot(np.ravel(data.time[n_sweep][0:n_block]), np.ravel(data.Eu_q[n_sweep][0:n_block]), ':r', linewidth=.5, label='Eu_q')
ax3.plot(np.ravel(data.time[n_sweep][0:n_block]), np.ravel(data.Ev_i[n_sweep][0:n_block]), '-b', linewidth=.5, label='Ev_i'); 
ax3.plot(np.ravel(data.time[n_sweep][0:n_block]), np.ravel(data.Ev_q[n_sweep][0:n_block]), ':r', linewidth=.5, label='Ev_q')
ax5.plot(np.ravel(data.time[n_sweep][0:n_block]), np.ravel(data.Ew_i[n_sweep][0:n_block]), '-b', linewidth=.5, label='Ew_i'); 
ax5.plot(np.ravel(data.time[n_sweep][0:n_block]), np.ravel(data.Ew_q[n_sweep][0:n_block]), ':r', linewidth=.5, label='Ew_q')
ax1.set_ylabel('Eu');  ax3.set_ylabel('Ev');  ax5.set_ylabel('Ew')
ax5.set_xlabel('Time (sec)')
#
title_label = '[JUICE/RPWI HF PSSR3-Rich (SID-23)]  ' + data_name + "  [" + date1 + "]";  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8)

# range: X-axis
"""
xlim=[0, 0.001]
ax1.set_xlim(xlim);  ax3.set_xlim(xlim);  ax5.set_xlim(xlim);  ax7.set_xlim(xlim)
"""
# range: Y-axis
"""
ylim=[-10**(p_raw_max-3.5), 10**(p_raw_max-3.5)]
ax1.set_ylim(ylim);  ax3.set_ylim(ylim);  ax5.set_ylim(ylim)
"""

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw-block.png'
    fig.savefig(png_fname)

# Get Spectrum

In [ ]:
# Spec mode:           0: low resolution  1; high resolution  
spec = juice_sid23.hf_sid23_getspec(data, band_mode)
power_str = juice_cdf.power_label(band_mode, unit_mode)
#
f_min = spec.freq[0]
f_max = spec.freq[n_length0-1]
d_freq = spec.freq[1]-spec.freq[0]

print(spec.EE.shape, n_time0, n_block0, n_length0)
print(f_min, "kHz -", f_max, "kHz  (Center:", data.freq_center[0], "kHz  df:", d_freq, "kHz")

# Spectrum - first

In [ ]:
n_sweep = 0

fig = plt.figure(figsize=(12, 10))
ax1 = fig.add_subplot(4, 1, 1);  ax4 = fig.add_subplot(4, 1, 2);  ax5 = fig.add_subplot(4, 1, 3);  ax6 = fig.add_subplot(4, 1, 4)

ax1.plot(spec.freq, np.ravel(spec.EE[n_sweep][0][:]), '-r', linewidth=.5, label='EuEu')
ax4.plot(spec.freq, np.ravel(spec.EuEu[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax5.plot(spec.freq, np.ravel(spec.EvEv[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax6.plot(spec.freq, np.ravel(spec.EwEw[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax1.plot(spec.freq, np.ravel(spec.EE[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='EvEv')
ax4.plot(spec.freq, np.ravel(spec.EuEu[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax5.plot(spec.freq, np.ravel(spec.EvEv[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax6.plot(spec.freq, np.ravel(spec.EwEw[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax1.plot(spec.freq, np.ravel(spec.EE[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='EwEw')
ax4.plot(spec.freq, np.ravel(spec.EuEu[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax5.plot(spec.freq, np.ravel(spec.EvEv[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax6.plot(spec.freq, np.ravel(spec.EwEw[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax1.set_ylabel('EE');  ax4.set_ylabel('EuEu'); ax5.set_ylabel('EvEv'); ax6.set_ylabel('EwEw'); ax6.set_xlabel('Frequency [kHz]')
ax1.set_yscale('log'); ax4.set_yscale('log');  ax5.set_yscale('log');  ax6.set_yscale('log')
#
date1 = data.epoch[n_sweep];  date1 = date1.strftime('First: %Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1;  ax1.set_title(title_date)
#
ax1.legend(loc='upper right', fontsize=8);  
ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8);   ax6.legend(loc='upper right', fontsize=8)

# range: Y-axis
ylim=[10**(p_raw_min-4), 10**(p_raw_max)]
ax1.set_ylim(ylim); ax4.set_ylim(ylim);  ax5.set_ylim(ylim); ax6.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec-1st.png'
    fig.savefig(png_fname)

# Peak
print("[First]")
peak_E = np.ravel(spec.EE[n_sweep][0]);   peak_f = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(spec.freq[peak_f]), "kHz")
peak_E = np.ravel(spec.EuEu[n_sweep][0]); peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(spec.freq[peak_f]), "kHz")
peak_E = np.ravel(spec.EvEv[n_sweep][0]); peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(spec.freq[peak_f]), "kHz")
peak_E = np.ravel(spec.EwEw[n_sweep][0]); peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(spec.freq[peak_f]), "kHz")

## Spectrum - Mid

In [ ]:
n_sweep = np.int16(n_time0/2)
n_sweep = n_time0-3
print("[mid]  SWEEP:", n_num0, "in total of", n_time0)

fig = plt.figure(figsize=(12, 10))
ax1 = fig.add_subplot(4, 1, 1);  ax4 = fig.add_subplot(4, 1, 2);  ax5 = fig.add_subplot(4, 1, 3);  ax6 = fig.add_subplot(4, 1, 4)

ax1.plot(spec.freq, np.ravel(spec.EE[n_sweep][0][:]), '-r', linewidth=.5, label='EuEu')
ax4.plot(spec.freq, np.ravel(spec.EuEu[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax5.plot(spec.freq, np.ravel(spec.EvEv[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax6.plot(spec.freq, np.ravel(spec.EwEw[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax1.plot(spec.freq, np.ravel(spec.EE[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='EvEv')
ax4.plot(spec.freq, np.ravel(spec.EuEu[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax5.plot(spec.freq, np.ravel(spec.EvEv[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax6.plot(spec.freq, np.ravel(spec.EwEw[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax1.plot(spec.freq, np.ravel(spec.EE[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='EwEw')
ax4.plot(spec.freq, np.ravel(spec.EuEu[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax5.plot(spec.freq, np.ravel(spec.EvEv[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax6.plot(spec.freq, np.ravel(spec.EwEw[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax1.set_ylabel('EE');  ax4.set_ylabel('EuEu'); ax5.set_ylabel('EvEv'); ax6.set_ylabel('EwEw'); ax6.set_xlabel('Frequency [kHz]')
ax1.set_yscale('log'); ax4.set_yscale('log');  ax5.set_yscale('log');  ax6.set_yscale('log')
#
date1 = data.epoch[n_sweep];  date1 = date1.strftime('Mid: %Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1;  ax1.set_title(title_date)
#
ax1.legend(loc='upper right', fontsize=8);  
ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8);   ax6.legend(loc='upper right', fontsize=8)

# range: Y-axis
ylim=[10**(p_raw_min-4), 10**(p_raw_max)]
ax1.set_ylim(ylim);  ax4.set_ylim(ylim);  ax5.set_ylim(ylim);  ax6.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec-mid.png'
    fig.savefig(png_fname)

# Peak
print("[Mid]")
peak_E = np.ravel(spec.EE[n_sweep][0]);   peak_f = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(spec.freq[peak_f]), "kHz")
peak_E = np.ravel(spec.EuEu[n_sweep][0]); peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(spec.freq[peak_f]), "kHz")
peak_E = np.ravel(spec.EvEv[n_sweep][0]); peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(spec.freq[peak_f]), "kHz")
peak_E = np.ravel(spec.EwEw[n_sweep][0]); peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(spec.freq[peak_f]), "kHz")

## Spectrum - Last

In [ ]:
n_sweep = n_time0-1

fig = plt.figure(figsize=(12, 10))
ax1 = fig.add_subplot(4, 1, 1);  ax4 = fig.add_subplot(4, 1, 2);  ax5 = fig.add_subplot(4, 1, 3);  ax6 = fig.add_subplot(4, 1, 4)

ax1.plot(spec.freq, np.ravel(spec.EE[n_sweep][0][:]), '-r', linewidth=.5, label='EuEu')
ax4.plot(spec.freq, np.ravel(spec.EuEu[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax5.plot(spec.freq, np.ravel(spec.EvEv[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax6.plot(spec.freq, np.ravel(spec.EwEw[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax1.plot(spec.freq, np.ravel(spec.EE[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='EvEv')
ax4.plot(spec.freq, np.ravel(spec.EuEu[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax5.plot(spec.freq, np.ravel(spec.EvEv[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax6.plot(spec.freq, np.ravel(spec.EwEw[n_sweep][n_block//2][:]), '-g', linewidth=.5, label='half')
ax1.plot(spec.freq, np.ravel(spec.EE[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='EwEw')
ax4.plot(spec.freq, np.ravel(spec.EuEu[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax5.plot(spec.freq, np.ravel(spec.EvEv[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax6.plot(spec.freq, np.ravel(spec.EwEw[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax1.set_ylabel('EE');  ax4.set_ylabel('EuEu'); ax5.set_ylabel('EvEv'); ax6.set_ylabel('EwEw'); ax6.set_xlabel('Frequency [kHz]')
ax1.set_yscale('log'); ax4.set_yscale('log');  ax5.set_yscale('log');  ax6.set_yscale('log')
#
date1 = data.epoch[n_sweep];  date1 = date1.strftime('Last: %Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1;  ax1.set_title(title_date)
#
ax1.legend(loc='upper right', fontsize=8);  
ax4.legend(loc='upper right', fontsize=8);  ax5.legend(loc='upper right', fontsize=8);   ax6.legend(loc='upper right', fontsize=8)

# range: Y-axis
ylim=[10**(p_raw_min-4), 10**(p_raw_max)]
ax1.set_ylim(ylim);  ax4.set_ylim(ylim);  ax5.set_ylim(ylim);  ax6.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec-last.png'
    fig.savefig(png_fname)

# Peak
print("[Last]")
peak_E = np.ravel(spec.EE[n_sweep][0]);   peak_f = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(spec.freq[peak_f]), "kHz")
peak_E = np.ravel(spec.EuEu[n_sweep][0]); peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(spec.freq[peak_f]), "kHz")
peak_E = np.ravel(spec.EvEv[n_sweep][0]); peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(spec.freq[peak_f]), "kHz")
peak_E = np.ravel(spec.EwEw[n_sweep][0]); peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(spec.freq[peak_f]), "kHz")

# Frequency - Time diagram

In [ ]:
p_min0 = p_min - 4
p_max0 = p_max
#
n_num = n_time0 * n_block0
EE_2d   = np.ravel(spec.EE);   EE_2d   = EE_2d.reshape(n_num, n_length0).transpose()
EuEu_2d = np.ravel(spec.EuEu); EuEu_2d = EuEu_2d.reshape(n_num, n_length0).transpose()
EvEv_2d = np.ravel(spec.EvEv); EvEv_2d = EvEv_2d.reshape(n_num, n_length0).transpose()
EwEw_2d = np.ravel(spec.EwEw); EwEw_2d = EwEw_2d.reshape(n_num, n_length0).transpose()
num_1d = np.arange(n_num)

fig2d = plt.figure(figsize=[12,12])
ax1 = fig2d.add_subplot(4, 1, 1);  ax2 = fig2d.add_subplot(4, 1, 2); ax3 = fig2d.add_subplot(4, 1, 3); ax4 = fig2d.add_subplot(4, 1, 4)

# Y-axis
ax1.set_ylim(f_min, f_max); ax2.set_ylim(f_min, f_max); ax3.set_ylim(f_min, f_max); ax4.set_ylim(f_min, f_max)
ax1.set_ylabel('f [kHz]');  ax2.set_ylabel('f [kHz]');  ax3.set_ylabel('f [kHz]');  ax4.set_ylabel('f [kHz]')

# X-axis
ax1.set_title("[" + data_name + "]  " + 'EE');  ax2.set_title('EuEu');  ax3.set_title('EvEv'); ax4.set_title('EwEw')
ax4.set_xlabel(str_date)

# Plot
p1 = ax1.pcolormesh(num_1d, spec.freq, EE_2d[:],   norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')
p2 = ax2.pcolormesh(num_1d, spec.freq, EuEu_2d[:], norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')
p3 = ax3.pcolormesh(num_1d, spec.freq, EvEv_2d[:], norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')
p4 = ax4.pcolormesh(num_1d, spec.freq, EwEw_2d[:], norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')

pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical"); pp1.set_label(power_str)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical"); pp2.set_label(power_str)
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical"); pp3.set_label(power_str)
pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical"); pp4.set_label(power_str)

# Plot
plt.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec-FT.png'
    fig2d.savefig(png_fname)

# Get Auto Correlation

In [ ]:
auto = juice_sid23.hf_sid23_getauto(data)

print(auto.EuEu.shape, n_time0, n_block0, n_length0)
print(f_min, "kHz -", f_max, "kHz  (Center:", data.freq_center[0], "kHz  df:", d_freq, "kHz")

## Auto First

In [ ]:
n_sweep = 0

fig = plt.figure(figsize=(12, 10))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(data.time[0][0], np.ravel(auto.EE[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax2.plot(data.time[0][0], np.ravel(auto.EuEu[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax3.plot(data.time[0][0], np.ravel(auto.EvEv[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax4.plot(data.time[0][0], np.ravel(auto.EwEw[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax1.plot(data.time[0][0], np.ravel(auto.EE[n_sweep][np.int16(n_block/2)][:]), '-g', linewidth=.5, label='half')
ax2.plot(data.time[0][0], np.ravel(auto.EuEu[n_sweep][np.int16(n_block/2)][:]), '-g', linewidth=.5, label='half')
ax3.plot(data.time[0][0], np.ravel(auto.EvEv[n_sweep][np.int16(n_block/2)][:]), '-g', linewidth=.5, label='half')
ax4.plot(data.time[0][0], np.ravel(auto.EwEw[n_sweep][np.int16(n_block/2)][:]), '-g', linewidth=.5, label='half')
ax1.plot(data.time[0][0], np.ravel(auto.EE[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax2.plot(data.time[0][0], np.ravel(auto.EuEu[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax3.plot(data.time[0][0], np.ravel(auto.EvEv[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax4.plot(data.time[0][0], np.ravel(auto.EwEw[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax1.set_ylabel('EE');  ax2.set_ylabel('EuEu'); ax3.set_ylabel('EvEv'); ax4.set_ylabel('EwEw'); ax6.set_xlabel('Time [sec]')
#
date1 = data.epoch[n_sweep];  date1 = date1.strftime('First: %Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1;  ax1.set_title(title_date)
#
ax1.legend(loc='upper right', fontsize=8);  
ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);   ax4.legend(loc='upper right', fontsize=8)

# range: X-axis
# xlim=[0, 0.0001];  ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim);  ax4.set_xlim(xlim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_auto-1st.png'
    fig.savefig(png_fname)

# Peak
peak_E = np.ravel(auto.EE[n_sweep][0][10:]);  peak_T = np.ravel(data.time[0][0][10:])
peak_t = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_t]), "at", '{:.2f}'.format(peak_T[peak_t]*1000), "msec")

## Auto - Mid

In [ ]:
n_sweep = np.int16(n_time0/2)
n_sweep = n_time0-3
print("[mid]  SWEEP:", n_num0, "in total of", n_time0)

fig = plt.figure(figsize=(12, 10))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(data.time[0][0], np.ravel(auto.EE[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax2.plot(data.time[0][0], np.ravel(auto.EuEu[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax3.plot(data.time[0][0], np.ravel(auto.EvEv[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax4.plot(data.time[0][0], np.ravel(auto.EwEw[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax1.plot(data.time[0][0], np.ravel(auto.EE[n_sweep][np.int16(n_block/2)][:]), '-g', linewidth=.5, label='half')
ax2.plot(data.time[0][0], np.ravel(auto.EuEu[n_sweep][np.int16(n_block/2)][:]), '-g', linewidth=.5, label='half')
ax3.plot(data.time[0][0], np.ravel(auto.EvEv[n_sweep][np.int16(n_block/2)][:]), '-g', linewidth=.5, label='half')
ax4.plot(data.time[0][0], np.ravel(auto.EwEw[n_sweep][np.int16(n_block/2)][:]), '-g', linewidth=.5, label='half')
ax1.plot(data.time[0][0], np.ravel(auto.EE[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax2.plot(data.time[0][0], np.ravel(auto.EuEu[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax3.plot(data.time[0][0], np.ravel(auto.EvEv[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax4.plot(data.time[0][0], np.ravel(auto.EwEw[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax1.set_ylabel('EE');  ax2.set_ylabel('EuEu'); ax3.set_ylabel('EvEv'); ax4.set_ylabel('EwEw'); ax6.set_xlabel('Time [sec]')
#
date1 = data.epoch[n_sweep];  date1 = date1.strftime('Mid: %Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1;  ax1.set_title(title_date)
#
ax1.legend(loc='upper right', fontsize=8);  
ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);   ax4.legend(loc='upper right', fontsize=8)

# range: X-axis
# xlim=[0, 0.0001]; ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim);  ax4.set_xlim(xlim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_auto-mid.png'
    fig.savefig(png_fname)

# Peak
peak_E = np.ravel(auto.EE[n_sweep][0][10:]);  peak_T = np.ravel(data.time[0][0][10:])
peak_t = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_t]), "at", '{:.2f}'.format(peak_T[peak_t]*1000), "msec")

## Auto - Last

In [ ]:
n_sweep = n_time0 - 1

fig = plt.figure(figsize=(12, 10))
ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)

ax1.plot(data.time[0][0], np.ravel(auto.EE[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax2.plot(data.time[0][0], np.ravel(auto.EuEu[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax3.plot(data.time[0][0], np.ravel(auto.EvEv[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax4.plot(data.time[0][0], np.ravel(auto.EwEw[n_sweep][0][:]), '-r', linewidth=.5, label='first')
ax1.plot(data.time[0][0], np.ravel(auto.EE[n_sweep][np.int16(n_block/2)][:]), '-g', linewidth=.5, label='half')
ax2.plot(data.time[0][0], np.ravel(auto.EuEu[n_sweep][np.int16(n_block/2)][:]), '-g', linewidth=.5, label='half')
ax3.plot(data.time[0][0], np.ravel(auto.EvEv[n_sweep][np.int16(n_block/2)][:]), '-g', linewidth=.5, label='half')
ax4.plot(data.time[0][0], np.ravel(auto.EwEw[n_sweep][np.int16(n_block/2)][:]), '-g', linewidth=.5, label='half')
ax1.plot(data.time[0][0], np.ravel(auto.EE[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax2.plot(data.time[0][0], np.ravel(auto.EuEu[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax3.plot(data.time[0][0], np.ravel(auto.EvEv[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax4.plot(data.time[0][0], np.ravel(auto.EwEw[n_sweep][n_block-1][:]), '-b', linewidth=.5, label='last')
ax1.set_ylabel('EE');  ax2.set_ylabel('EuEu'); ax3.set_ylabel('EvEv'); ax4.set_ylabel('EwEw'); ax6.set_xlabel('Time [sec]')
#
date1 = data.epoch[n_sweep];  date1 = date1.strftime('Last: %Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1;  ax1.set_title(title_date)
#
ax1.legend(loc='upper right', fontsize=8);  
ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8);   ax4.legend(loc='upper right', fontsize=8)

# range: X-axis
# xlim=[0, 0.0001]; ax1.set_xlim(xlim);  ax2.set_xlim(xlim);  ax3.set_xlim(xlim);  ax4.set_xlim(xlim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_auto-last.png'
    fig.savefig(png_fname)

# Peak
peak_E = np.ravel(auto.EE[n_sweep][0][10:]);  peak_T = np.ravel(data.time[0][0][10:])
peak_t = np.argmax(peak_E); print("Peak EE  :", '{:.2e}'.format(peak_E[peak_t]), "at", '{:.2f}'.format(peak_T[peak_t]*1000), "msec")

## Auto - Time

In [ ]:
n_num = n_time0 * n_block0
EE_2d   = np.ravel(auto.EE);    EE_2d   = EE_2d.reshape(n_num, n_length0).transpose()
EuEu_2d = np.ravel(auto.EuEu);  EuEu_2d = EuEu_2d.reshape(n_num, n_length0).transpose()
EvEv_2d = np.ravel(auto.EvEv);  EvEv_2d = EvEv_2d.reshape(n_num, n_length0).transpose()
EwEw_2d = np.ravel(auto.EwEw);  EwEw_2d = EwEw_2d.reshape(n_num, n_length0).transpose()
num_1d = np.arange(n_num)

fig2d = plt.figure(figsize=[12,12])
ax1 = fig2d.add_subplot(4, 1, 1);  ax2 = fig2d.add_subplot(4, 1, 2);  ax3 = fig2d.add_subplot(4, 1, 3);  ax4 = fig2d.add_subplot(4, 1, 4)

# Y-axis
ax1.set_ylabel('time [sec]');  ax2.set_ylabel('time [sec]');  ax3.set_ylabel('time [sec]');  ax4.set_ylabel('time [sec]')

# X-axis
ax1.set_title("[" + data_name + "]  " + 'EE');  ax2.set_title('EuEu');  ax3.set_title('EvEv'); ax4.set_title('EwEw')
ax4.set_xlabel(str_date)

# Plot
p1 = ax1.pcolormesh(num_1d, data.time[0][0], EE_2d[:], cmap='jet')
p2 = ax2.pcolormesh(num_1d, data.time[0][0], EuEu_2d[:], cmap='jet')
p3 = ax3.pcolormesh(num_1d, data.time[0][0], EvEv_2d[:], cmap='jet')
p4 = ax4.pcolormesh(num_1d, data.time[0][0], EwEw_2d[:], cmap='jet')

pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label('auto-corr')
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical");  pp2.set_label('auto-corr')
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical");  pp3.set_label('auto-corr')
pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical");  pp4.set_label('auto-corr')

# Plot
plt.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_auto-FT.png'
    fig2d.savefig(png_fname)